In [ ]:
import hyperspy.api as hs
import numpy as np
import matplotlib.pyplot as plt
from utils.get_raw_data import locate_raw_data

emsa = locate_raw_data()




In [ ]:
%matplotlib qt

In [ ]:
ss = hs.load(emsa[7], signal_type="EDS_TEM")
ss.add_elements(["As", "Ga"])
ss.metadata

In [ ]:
ss.metadata.General.title = "GaAs bulk, 30 kV"
# update the xaxis and yaxis labels
ss.axes_manager
ss.axes_manager[0].name = "Energy"

In [ ]:
ss.plot(xray_lines=True)

In [ ]:
m = ss.create_model(auto_background=True)
m.fit()
m.plot(True)

In [ ]:
m.active_components[1].A.value

In [ ]:
m.active_components[1].name

In [ ]:
sm = hs.signals.Signal1D(np.zeros(2048))
sm.axes_manager[0].offset = ss.axes_manager[0].offset
sm.axes_manager[0].scale = ss.axes_manager[0].scale

mg = sm.create_model()
g = hs.model.components1D.Gaussian()
mg.append(g)
gau = []

for i in range(1, len(m.active_components)):
        g.centre.value = m.active_components[i].centre.value
        g.sigma.value = m.active_components[i].sigma.value
        g.A.value = m.active_components[i].A.value /97
        gs = mg.as_signal()
        name = f'{m.active_components[i].name:<8}'
        name+= f'fwhm={m.active_components[i].fwhm*1000:<5.0f}'
        name+= f'μ={m.active_components[i].centre.value:<7.2f}'
        name+= f'A={m.active_components[i].A.value:<8.0}'
        name+= f'σ={m.active_components[i].sigma.value*1000:.0f}'
        gs.metadata.General.title = name
        # print(g.A.value)
        gau.append(gs)

gau.append(ss)
gau.append(m.as_signal())


fig, ax = plt.subplots(1,1, figsize=(20,12))
plt.rcParams['font.size'] = '16'
plt.rcParams['lines.linewidth'] = 3
plt.rcParams['font.family'] = 'monospace'

hs.plot.plot_spectra(gau,legend='auto', ax=ax, fig=fig, style='overlap')

plt.title('Gaussian curves are scaled with 1/97, I do not know why this is needed')

# y scale in log
plt.yscale('log')
# lim ymin at 50
plt.ylim(ymin=50)

plt.show()




In [ ]:
plt.close('all')

In [ ]:
sL = ss.isig[0.8:1.5]
sL.plot(xray_lines=True)

In [ ]:
sK = ss.isig[8.5:12.5]
sK.plot(xray_lines=True)

In [ ]:
sGK = ss.isig[9.0:9.5]
sGK.plot(xray_lines=True)

In [ ]:
plt.close('all')